# Video Generation Pipeline

This notebook generates a video from a given topic or text using Ollama for scripting, PIL for slides, Edge-TTS for audio, and MoviePy for assembly.

In [10]:
import json
import os
import textwrap
import requests
import asyncio
from PIL import Image, ImageDraw, ImageFont
import edge_tts
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips, VideoFileClip

In [11]:
# Configuration
OLLAMA_API_URL = "http://localhost:11434/api/generate"
# Use 'llama3' or another model you have installed
OLLAMA_MODEL = "llama3"

OUTPUT_DIR = "output"
SCENE_DIR = os.path.join(OUTPUT_DIR, "scenes")
AUDIO_DIR = os.path.join(OUTPUT_DIR, "audio")
FINAL_VIDEO_DIR = os.path.join(OUTPUT_DIR, "video")

os.makedirs(SCENE_DIR, exist_ok=True)
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(FINAL_VIDEO_DIR, exist_ok=True)

## Step 1: Script Generation with Ollama

In [12]:
def generate_script(topic):
    prompt = f"""
    Convert this topic into a structured video plan.
    Topic: {topic}
    Return JSON only:
    {{
      "scenes": [
        {{
          "title": "",
          "bullets": [],
          "narration": ""
        }}
      ]
    }}
    """
    
    print(f"Generating script for: {topic}...")
    try:
        response = requests.post(OLLAMA_API_URL, json={
            "model": OLLAMA_MODEL,
            "prompt": prompt,
            "format": "json",
            "stream": False
        })
        response.raise_for_status()
        return json.loads(response.json()['response'])
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to Ollama: {e}")
        return None
    except json.JSONDecodeError:
        print("Error decoding JSON response from Ollama.")
        return None

## Step 2: Slide Generation (PIL)

In [13]:
def create_slide(scene, index):
    width, height = 1280, 720
    # White background (you can change to gradient code if desired)
    img = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(img)
    
    # Attempt to load a nice font, fallback to default
    try:
        # Windows usually has arial
        title_font = ImageFont.truetype("arial.ttf", 70)
        text_font = ImageFont.truetype("arial.ttf", 40)
    except:
        title_font = ImageFont.load_default()
        text_font = ImageFont.load_default()
        
    # Draw Title
    title_text = scene.get('title', f"Scene {index}")
    draw.text((50, 50), title_text, fill='black', font=title_font)
    
    # Draw Bullets
    y = 180
    bullets = scene.get('bullets', [])
    for bullet in bullets:
        # Wrap text
        lines = textwrap.wrap(bullet, width=50)
        for line in lines:
            draw.text((80, y), f"• {line}", fill='black', font=text_font)
            y += 50
            
    # Save slide
    filename = os.path.join(SCENE_DIR, f"scene_{index}.png")
    img.save(filename)
    return filename

## Step 3: Audio Generation (Edge-TTS)

In [14]:
async def generate_audio(text, index):
    # Voices: en-US-ChristopherNeural, en-US-AriaNeural, etc.
    voice = "en-US-ChristopherNeural"
    output_file = os.path.join(AUDIO_DIR, f"scene_{index}.mp3")
    
    print(f"Generating audio for scene {index}...")
    try:
        communicate = edge_tts.Communicate(text, voice)
        await communicate.save(output_file)
        return output_file
    except Exception as e:
        print(f"Error generating audio: {e}")
        return None

## Step 4: Video Assembly (MoviePy)
Uses MoviePy to combine image and audio into a video clip.

In [15]:
def create_video_clip(image_path, audio_path, index):
    output_path = os.path.join(FINAL_VIDEO_DIR, f"scene_{index}.mp4")
    
    print(f"Creating video clip for scene {index} using MoviePy...")
    try:
        # Load Audio
        audio_clip = AudioFileClip(audio_path)
        # Create Image Clip with duration of audio
        video_clip = ImageClip(image_path).set_duration(audio_clip.duration)
        # Set Audio
        video_clip = video_clip.set_audio(audio_clip)
        # Write File
        video_clip.write_videofile(output_path, fps=24, codec='libx264', audio_codec='aac')
        
        return output_path
    except Exception as e:
        print(f"MoviePy failed for scene {index}: {e}")
        return None

## Step 5: Merge All Scenes

In [16]:
def merge_scenes(video_files):
    output_filename = "final_video.mp4"
    
    print("Merging all scenes into final video...")
    try:
        # Create VideoFileClip objects for each file
        clips = [VideoFileClip(f) for f in video_files]
        
        final_clip = concatenate_videoclips(clips)
        final_clip.write_videofile(output_filename, fps=24, codec='libx264', audio_codec='aac')
        print(f"Done! Output: {output_filename}")
        return output_filename
    except Exception as e:
        print(f"Error merging scenes: {e}")
        return None

## Execution Pipeline

In [17]:
async def main(topic):
    # 1. Generate Script
    script_data = generate_script(topic)
    if not script_data:
        return
    
    # Save plan for reference
    with open("video_plan.json", "w") as f:
        json.dump(script_data, f, indent=2)
    
    scenes = script_data.get('scenes', [])
    video_clips = []
    
    for i, scene in enumerate(scenes, 1):
        print(f"Processing Scene {i}: {scene.get('title')}")
        
        # 2. Tools -> Image
        img_path = create_slide(scene, i)
        
        # 3. Narration -> Audio
        narration = scene.get('narration', '')
        if not narration:
            print(f"Warning: No narration for scene {i}")
            continue
            
        audio_path = await generate_audio(narration, i)
        if not audio_path:
            continue
            
        # 4. Combine -> Clip
        clip_path = create_video_clip(img_path, audio_path, i)
        if clip_path:
            video_clips.append(clip_path)
            
    # 5. Merge
    if video_clips:
        merge_scenes(video_clips)
    else:
        print("No video clips were created.")

# Example Usage
# await main("The history of the internet")